## Documentation

- API Guide can be found in this link https://ib-insync.readthedocs.io/api.html
- Options Example (SPX) with ib_insync https://nbviewer.org/github/erdewit/ib_insync/blob/master/notebooks/option_chain.ipynb


In [ ]:
from ib_insync import *
import pandas as pd
util.startLoop()
from datetime import date
from datetime import datetime

def verify_daycount(dates):
    ds = []
    today = date.today()
    today = datetime.strftime(today, "%Y%m%d")
    today = datetime.strptime(today, "%Y%m%d")
    for adate in dates:
        date_obj = datetime.strptime(adate, "%Y%m%d")
        days_diff = (date_obj - today).days
        if (38 < days_diff < 53) or (90 < days_diff < 150):
            ds.append(adate)
    return ds
    

ib = IB()
ib.connect('127.0.0.1', 7496, clientId = 7)

tick = 'AAPL'
print( " >>> DEFINING STOCK ")
stk = Stock(tick, 'SMART', 'USD')
ib.qualifyContracts(stk)
ib.reqMarketDataType(4) #wait a bit
chains = ib.reqSecDefOptParams(stk.symbol, '', stk.secType, stk.conId)
ds_chains = pd.DataFrame(chains)
expirations = sorted(ds_chains['expirations'].iloc[0])
expirations = verify_daycount(expirations) #convert expiration day to number of days left to expire
strikes = ds_chains['strikes'].iloc[0]
contractID = ds_chains['underlyingConId'].iloc[0]
rights = ['P', 'C'] #easier for algirthm built # P = Put ; C = Call

#organize all Option Contracts into data set
print( " >>> DEFINING OPTIONS ")
contracts = [Option(tick, ex, s, r, 'SMART') for r in rights for ex in expirations for s in strikes]
print( " >>> QUALIFYING OPTIONS ")
qualifiedContracts = ib.qualifyContracts(*contracts)
print( " >>> RETRIEVING DETAILS ")
qualifiedContractDetails = ib.reqTickers(*qualifiedContracts)
ds = pd.DataFrame()
print ( " >>> GETTING GREEKS AND OPTIONS INFO")
for qcd in qualifiedContractDetails:
    try:
        details = qcd.contract
        exp, strike, right = details.lastTradeDateOrContractMonth, details.strike, details.right
        greek = qcd.modelGreeks
        delta, theta, gamma, vega, pvDiv, optPrice, impliedVol = greek.delta, greek.theta, greek.gamma, greek.vega, greek.pvDividend, greek.optPrice, greek.impliedVol
        row = {
            'Symbol': tick, 'Expiration': exp , 'Strike': strike, 'Right': right,
            'Delta': delta, 'Theta': theta, 'Vega': vega, 'Gamma': gamma,
            'impVol': impliedVol, 'optPrice': optPrice, 'Ask': qcd.ask, 'Bid': qcd.bid, 'Last': qcd.last}
        ds = pd.concat([ds, pd.DataFrame([row])], axis = 0, ignore_index = True)
    except:
        print ( " >>> NOGOOD CONTRACT")

#next: call all options during market time to get greek values
def days_diff(adate):
    today = date.today()
    today = datetime.strftime(today, "%Y%m%d")
    today = datetime.strptime(today, "%Y%m%d")
    date_obj = datetime.strptime(adate, "%Y%m%d")
    return (date_obj - today).days

ds['daysdiff'] = list(map(days_diff, ds['Expiration'])) 
ds['Delta'] = abs(ds['Delta'])
ds['Theta'] = abs(ds['Theta'])
print(ds)

# ds_p1 = ds[(0.1 <= ds['Delta'] <= 0.2) & (ds['Right'] == 'P') & (days_diff(ds['Expiration'], 90, 150))]
ds_p1 = ds[(ds['Right'] == 'P') & (ds['daysdiff'] >= 90) & (ds['daysdiff'] <= 150) & (ds['Delta'] >= 0.1) & (ds['Delta'] <= 0.2)]
ds_p1['REGEL'] = ['P1'] * len(ds_p1)
ds_p2 = ds[(ds['Right'] == 'P') & (ds['daysdiff'] >= 38) & (ds['daysdiff'] <= 53) & (ds['Delta'] >= 0.2) & (ds['Delta'] <= 0.3)]
ds_p2['REGEL'] = ['P2'] * len(ds_p2)
ds_c3 = ds[(ds['Right'] == 'C') & (ds['daysdiff'] >= 38) & (ds['daysdiff'] <= 53) & (ds['Delta'] >= 0.2) & (ds['Delta'] <= 0.3)]
ds_c3['REGEL'] = ['C3'] * len(ds_c3)
ds_c4 = ds[(ds['Right'] == 'C') & (ds['daysdiff'] >= 90) & (ds['daysdiff'] <= 150) & (ds['Delta'] >= 0.1) & (ds['Delta'] <= 0.2)]
ds_c4['REGEL'] = ['C4'] * len(ds_c4)

main = pd.concat([ds_p1, ds_p2, ds_c3, ds_c4], ignore_index=True)
def p_marg(p1, p2):
    if (p1 == p2 - 10) | (p1 == p2 - 20):
        return True
    return False

def c_marg(c3, c4):
    if (c4 == c3 + 10) | (c4 == c3 + 20):
        return True
    return False

def p_theta(p1, p2):
    if (p2 > p1 * 1.5):
        return True
    return False

def c_theta(c3, c4):
    if (c3 > c4 * 1.5):
        return True
    return False

def bundle(ds):
    p1 = ds[ds['REGEL'] == 'P1']
    p2 = ds[ds['REGEL'] == 'P2']
    c3 = ds[ds['REGEL'] == 'C3']
    c4 = ds[ds['REGEL'] == 'C4']
    ps = []
    cs = []
    i = 0

    #convert two loops to function prperly (rearrange functions in sheet)
    for i in range(len(p1)):
        j = 0
        for j in range(len(p2)):
            # code begin
            if p_marg(p1.iloc[i]['Strike'], p2.iloc[j]['Strike']) & p_theta(p1.iloc[i]['Theta'], p2.iloc[j]['Theta']):
                ps.append((i, j))
            # code end
            j += 1
        i += 1

    for i in range(len(c3)):
        j = 0
        for j in range(len(c4)):
            # code begin
            if c_marg(c3.iloc[i]['Strike'], c4.iloc[j]['Strike']) & c_theta(c3.iloc[i]['Theta'], c4.iloc[j]['Theta']):
                cs.append((i, j))
            # code end
            j += 1
        i += 1
    

    legs_indx = [(p1, p2, c3, c4) for (p1, p2) in ps for (c3, c4) in cs]

    dss = []
    for (a, b, c, d) in legs_indx:
        ds = pd.concat([p1.iloc[a].to_frame().T, p2.iloc[b].to_frame().T, c3.iloc[c].to_frame().T, c4.iloc[d].to_frame().T], ignore_index=True)
        # df = pd.DataFrame.from_records(ds)
        dss.append(ds)
    
    return dss


bundles = bundle(main)
pd.concat(bundles).to_csv(tick+'_options.csv')
ib.disconnect()


In [49]:
# Create the main line at each bundle sheet
ib.connect('127.0.0.1', 7496, clientId = 8)
stk = Stock(tick, 'SMART', 'USD')
qc = ib.qualifyContracts(stk)
data = ib.reqMktData(stk)
print(data)
ib.disconnect()
d = " "
mids = {
'Symbol': tick, 
'Last': d,
'Change%': d,
'Low-26W': d,
'High-26W': d,
"IV%": d,
"H-IV-30%": d,
"H-IV-Mean%": d,
"RSI-14D": d
}

mids = pd.DataFrame(mids)


AttributeError: 'list' object has no attribute 'secType'

In [50]:
import openpyxl
import xlsxwriter
# bundles.to_excel('pandas_to_excel.xlsx', sheet_name='new_sheet_name')

writer = pd.ExcelWriter('bundles.xlsx',engine='xlsxwriter')
workbook = writer.book
worksheet = workbook.add_worksheet('Result')
writer.sheets['Bundles'] = worksheet


bundle_count = 1
currow, curcol = 4, 0
for ds in bundles:
        symbol = str(list(ds[ds['REGEL'] == 'P1']['Symbol'])[0])
        worksheet.write_string(currow, 0, str(symbol) + ": Bundle " + str(bundle_count))
        currow += 1
        ds['Change'] = ((ds['Strike'] - ds['Last'])/ds['Last']) #convert to percentage
        ds['Mark'] = (ds['Ask'] + ds['Bid']) / 2
        ds['impVol'] = ds['impVol'] * 100
        rtd_p, rtd_c = list(ds[ds['REGEL'] == 'P2']['Theta'])[0] - list(ds[ds['REGEL'] == 'P1']['Theta'])[0], list(ds[ds['REGEL'] == 'C3']['Theta'])[0] - list(ds[ds['REGEL'] == 'C4']['Theta'])[0]
        ds['RTD'] = [rtd_p, rtd_p, rtd_c, rtd_c]
        ds['RTM'] = ds['RTD'] * 22
        ds['RTY'] = ds['RTM'] * 12
        mp, mc = list(ds[ds['REGEL'] == 'P2']['Strike'])[0] - list(ds[ds['REGEL'] == 'P1']['Strike'])[0], list(ds[ds['REGEL'] == 'C4']['Strike'])[0] - list(ds[ds['REGEL'] == 'C3']['Strike'])[0]
        ds['Margin'] = [mp, mp, mc, mc]

        ds[['REGEL', 'Expiration', 'daysdiff', 'Strike', 'Change', 'Mark', 'Delta', 'Theta', 'Vega', 'impVol', 'RTM']].to_excel(writer, sheet_name = 'Bundles', startrow = currow, startcol = 0)
        currow += ds.shape[0] + 2

        print(ds)

        pnative = {
        'Type': "P-Native", 
        'Margin': mp,
        'Monthly%': rtd_p * 22,
        'Yearly%': (rtd_p * 22 * 12 * 100) / mp,
        'Yearly$': rtd_p * 22 * 12,
        "Cost": list(ds[ds['REGEL'] == 'P1']['Mark'])[0] + list(ds[ds['REGEL'] == 'P2']['Mark'])[0]
        }

        cnative = {
        'Type': "C-Native", 
        'Margin': mc,
        'Monthly%': rtd_c * 22,
        'Yearly%': (rtd_c * 22 * 12 * 100) / mc,
        'Yearly$': (rtd_c * 22 * 12),
        "Cost": list(ds[ds['REGEL'] == 'C3']['Mark'])[0] + list(ds[ds['REGEL'] == 'C4']['Mark'])[0]
        }

        posnative = {
        'Type': "POS-Native", 
        'Margin': max(pnative['Margin'], cnative['Margin']),
        'Monthly%': (rtd_c * 22 + rtd_p * 22) / max(mp, mc),
        'Yearly%': ((rtd_c * 22 * 12) + (rtd_p * 22 * 12)) / max(mp, mc),
        'Yearly$': (pnative['Yearly$'] + cnative['Yearly$']),
        "Cost": ds['Mark'].sum()
        }

        ds_ = pd.DataFrame([pnative, cnative, posnative])
        ds_.to_excel(writer, sheet_name = 'Bundles', startrow = currow, startcol = 0)
        currow += 7 # 4 for the native ds_, 2 for spacing
        bundle_count += 1

writer.save() #save the excelt sheet

  Symbol Expiration Strike Right     Delta     Theta      Vega     Gamma  \
0   AAPL   20221021  115.0     P  0.101448    0.0289  0.138848  0.005547   
1   AAPL   20220826  135.0     P  0.226517  0.072055  0.142327  0.015768   
2   AAPL   20220826  160.0     C  0.222464  0.057609  0.136213  0.020562   
3   AAPL   20221021  170.0     C  0.194905  0.033372  0.223164  0.012788   

                                   impVol  optPrice  ...   Bid  Last daysdiff  \
0  427064762071291482716242088845901824.0  1.705095  ...   1.7  1.72       95   
1  398079817238595389607636289447788544.0   2.68992  ...  2.65  2.71       39   
2  301868356163949886088091451086864384.0  1.781143  ...  1.78  1.83       39   
3  287910991966858868089969012899840000.0  2.190441  ...   2.2   2.2       95   

  REGEL     Change   Mark       RTD  Margin       RTM        RTY  
0    P1  65.860465   1.72  0.043154    20.0  0.949397  11.392766  
1    P2  48.815498  2.695  0.043154    20.0  0.949397  11.392766  
2    C3  86.

In [ ]:
[Ticker(
    contract=Option(symbol='SPY', lastTradeDateOrContractMonth='20180420', strike=270, right='C', exchange='SMART'), 
    time=datetime.datetime(2018, 3, 1, 16, 1, 17, 467516, tzinfo=datetime.timezone.utc), 
    bid=6.83, bidSize=1156, ask=6.88, askSize=11, last=6.88, lastSize=1, volume=2264, high=7.67, low=6.17, close=6.95, ticks=[], tickByTicks=[], domBids=[], domAsks=[], domTicks=[], 
    bidGreeks=OptionComputation(impliedVol=0.160298280414867, delta=None, optPrice=6.829999923706055, pvDividend=1.0992683616305827, gamma=None, vega=None, theta=None, undPrice=271.32000732421875), 
    askGreeks=OptionComputation(impliedVol=0.16155485178197637, delta=None, optPrice=6.880000114440918, pvDividend=1.0992683616305827, gamma=None, vega=None, theta=None, undPrice=271.32000732421875), 
    lastGreeks=OptionComputation(impliedVol=None, delta=None, optPrice=6.880000114440918, pvDividend=1.0992683616305827, gamma=None, vega=None, theta=None, undPrice=271.32000732421875), 
    modelGreeks=OptionComputation(impliedVol=0.15770592183303841, delta=0.5344203452034115, optPrice=6.665772690114447, pvDividend=1.0992683616305827, gamma=0.026063908021373355, vega=0.3941542814583521, theta=-0.07095144483744885, undPrice=271.32000732421875)
)]
